In [1]:
import tensorflow as tf
import pandas as pd
import string
import requests
import re

In [2]:
response=requests.get("https://sncuvobxpqxmfhmfjnlklr.coursera-apps.org/edit/Week%201/Dinosaur%20Island%20--%20Character-level%20language%20model/shakespeare.txt#")

In [2]:
data=open("shakespeare.txt","r")

In [3]:

data=data.read()

In [4]:
data=data.split("\n")

In [5]:
len(data)

2469

In [6]:
data=" ".join(data)

In [7]:
def clean(text):
    text = text.lower()
    text = re.sub('\[.*?\,\']', ' ', text)
    text = re.sub('https?://\S+|www\.\S+', ' ', text)
    text = re.sub('<.*?>+', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    text=text.split()
    
    
    return text

In [8]:
data=clean(data)
len((data))

17976

In [13]:
data=data[5:]

In [71]:
len(set(data))

3086

In [17]:
window=31
lines=[]
for i in range(len(data)-window-1):
  line=data[i:i+window]
  line=" ".join(line)
  lines.append(line)
  

In [43]:
lines[0]

'from fairest creatures we desire increase that thereby beauty s rose might never die but as the riper should by time decease his tender heir might bear his memory but thou'

In [24]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [25]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(lines)
sequences=tokenizer.texts_to_sequences(lines)

In [26]:
len(sequences)

17939

In [27]:
sequences=np.array(sequences)

In [28]:
sequences.shape

(17939, 31)

In [46]:
sequences.shape

(17971,)

In [29]:
x,y=sequences[:,:-1],sequences[:,-1]

In [33]:
vocab_size=len(tokenizer.index_word)+1
vocab_size

3084

In [31]:
y=to_categorical(y,num_classes=vocab_size)

In [32]:
y.shape

(17939, 3084)

In [34]:
seq_length=x.shape[1]

In [35]:
seq_length

30

##LSTM

In [38]:
model=Sequential()
model.add(Embedding(input_dim=vocab_size,output_dim=300,input_length=seq_length))
#model.add(Embedding(vocab_size,40,input_length=seq_length))
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100,activation="relu"))
model.add(Dense(vocab_size,activation='softmax'))

In [39]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 300)           925200    
_________________________________________________________________
lstm_2 (LSTM)                (None, 30, 100)           160400    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 3084)              311484    
Total params: 1,487,584
Trainable params: 1,487,584
Non-trainable params: 0
_________________________________________________________________


In [41]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x, y, batch_size=100, epochs=100)

Epoch 1/100
180/180 [==============================] - 7s 39ms/step - loss: 0.9349 - accuracy: 0.7590
Epoch 2/100
180/180 [==============================] - 7s 39ms/step - loss: 0.8160 - accuracy: 0.8018
Epoch 3/100
180/180 [==============================] - 7s 39ms/step - loss: 0.8127 - accuracy: 0.8009
Epoch 4/100
180/180 [==============================] - 7s 38ms/step - loss: 0.8051 - accuracy: 0.8027
Epoch 5/100
180/180 [==============================] - 7s 38ms/step - loss: 0.7846 - accuracy: 0.8073
Epoch 6/100
180/180 [==============================] - 7s 39ms/step - loss: 0.7697 - accuracy: 0.8107
Epoch 7/100
180/180 [==============================] - 7s 37ms/step - loss: 0.7568 - accuracy: 0.8145
Epoch 8/100
180/180 [==============================] - 7s 37ms/step - loss: 0.7404 - accuracy: 0.8153
Epoch 9/100
180/180 [==============================] - 7s 38ms/step - loss: 0.7166 - accuracy: 0.8194
Epoch 10/100
180/180 [==============================] - 7s 38ms/step - loss: 0.686

In [78]:
def generate_text(model,tokenizer,text_seq_length,seed_text,n_words):
  text=[]
  for i in range(n_words):
   
    encoded=tokenizer.texts_to_sequences([seed_text])[0]
    encoded=pad_sequences([encoded],maxlen=text_seq_length,truncating='pre')
    #y_predict=model.predict_classes(encoded)
    y_predict=np.argmax(model.predict(encoded), axis=-1)
    predicted_word=''
    for word,index in tokenizer.word_index.items():
      if y_predict==index:
        predicted_word=word
        break

    seed_text=seed_text+' '+predicted_word

    text.append(predicted_word)
    

  return ' '.join(text)


In [80]:
seed=lines[2]
seed

'creatures we desire increase that thereby beauty s rose might never die but as the riper should by time decease his tender heir might bear his memory but thou contracted to'

In [81]:
generate_text(model,tokenizer,seq_length,seed,20)

'thine own bright eyes feed st thy light s flame with self substantial fuel making a famine where abundance lies'